## Imports and read DF

In [59]:
import pandas as pd
import numpy as np
import json
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
import re
from sentence_transformers import SentenceTransformer
import torch
import faiss

In [41]:
file=json.load(open('Market Research Report_extracted_text.json'))
file
print(len(file))
df= pd.DataFrame(file, columns=['text'])
df

9


,text
0,\n \nMARKET RESEARCH \nREPORT: ANALYSIS OF \...
1,\nIntroduction \nThis market research report...
2,\nExecutive Summary \nThis section provides ...
3,\n4. Merge PDF: Enable merging multiple PDFs...
4,\n• Process Modes \n1. Professional Transl...
5,\nResults \nComparison Table \n \n \n \n \...
6,\nPricing Plans for Paid Tool s \nTool Plan ...
7,\n \nDetailed Findings \nDoctranslator \n• ...
8,\nTranslaDocs \n• Cons: \no No OCR support...


## Text Preprocessing

In [42]:
df

,text
0,\n \nMARKET RESEARCH \nREPORT: ANALYSIS OF \...
1,\nIntroduction \nThis market research report...
2,\nExecutive Summary \nThis section provides ...
3,\n4. Merge PDF: Enable merging multiple PDFs...
4,\n• Process Modes \n1. Professional Transl...
5,\nResults \nComparison Table \n \n \n \n \...
6,\nPricing Plans for Paid Tool s \nTool Plan ...
7,\n \nDetailed Findings \nDoctranslator \n• ...
8,\nTranslaDocs \n• Cons: \no No OCR support...


In [43]:
def clean_text(text):
    text = str(text)  # Ensure it's a string
    text = re.sub(r'\n{3,}', '\n\n', text)  # 3+ newlines → 2 newlines
    text = re.sub(r'\s+', ' ', text)        # Remove extra spaces
    return text.strip()

df['clean_text'] = df['text'].apply(clean_text)


In [44]:
df

,text,clean_text
0,\n \nMARKET RESEARCH \nREPORT: ANALYSIS OF \...,MARKET RESEARCH REPORT: ANALYSIS OF DOCUMENT T...
1,\nIntroduction \nThis market research report...,Introduction This market research report analy...
2,\nExecutive Summary \nThis section provides ...,Executive Summary This section provides a high...
3,\n4. Merge PDF: Enable merging multiple PDFs...,4. Merge PDF: Enable merging multiple PDFs int...
4,\n• Process Modes \n1. Professional Transl...,• Process Modes 1. Professional Translation: T...
5,\nResults \nComparison Table \n \n \n \n \...,Results Comparison Table Tool Layout Preservat...
6,\nPricing Plans for Paid Tool s \nTool Plan ...,Pricing Plans for Paid Tool s Tool Plan Name C...
7,\n \nDetailed Findings \nDoctranslator \n• ...,Detailed Findings Doctranslator • Pros: o Free...
8,\nTranslaDocs \n• Cons: \no No OCR support...,TranslaDocs • Cons: o No OCR support . o No Ar...


## Chunking 

In [45]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=lambda x: len(x.split()),
    separators=["\n\n\n", "\n\n", "\n", ". ", "! ", "? ", "; ", ", ", " ", ""],
    keep_separator=False,
    add_start_index=True,
    strip_whitespace=True
)

In [46]:
combined_text = "\n".join(df['clean_text'].tolist())
print(combined_text)

MARKET RESEARCH REPORT: ANALYSIS OF DOCUMENT TRANSLATION TOOLS Evaluating Leading Solutions for Multilingual Document Translation Mah inour Mohammad
Introduction This market research report analyzes competitors offering document translation tools that support PDF, Word, Excel, and scanned images while preserving layout and formatting. The focus is on tools that handle Arabic, French, and English languages, catering to both B2B and B2C markets. The key features evaluated include layout preservation, Arabic support and quality, translation accuracy and speed, pricing model, and Optical Character Recognition (OCR) support. To assess these tools, a series of test cases were conducted for each language, including: 1. Text -based documents: Evaluating basic translation accuracy, layout preservation, handling of number lists, bullet points, and right -to-left (RTL) and left -to-right (LTR) conversions. 2. Scanned documents: Testing OCR performance, particularly for Arabic, and preservation of

In [47]:
chunks = text_splitter.split_text(combined_text)
print(f"Created {len(chunks)} chunks")

Created 16 chunks


In [48]:
type(chunks)

list

## Embeddings

In [84]:
device= 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',device=device,cache_folder='embedder_model_cache')

In [85]:
chunks_embed = {}
chunks_text = {}

for i, chunk in enumerate(chunks):
    chunks_text[i] = chunk
    chunks_embed[i] = model.encode(chunk,convert_to_numpy=True, normalize_embeddings=True)


chunks_embed = {i: model.encode(chunk) for i, chunk in enumerate(chunks)}
chunk_embedding = chunks_embed[0] 

In [86]:
chunks_text

{0: 'MARKET RESEARCH REPORT: ANALYSIS OF DOCUMENT TRANSLATION TOOLS Evaluating Leading Solutions for Multilingual Document Translation Mah inour Mohammad',
 1: 'Introduction This market research report analyzes competitors offering document translation tools that support PDF, Word, Excel, and scanned images while preserving layout and formatting. The focus is on tools that handle Arabic, French, and English languages, catering to both B2B and B2C markets. The key features evaluated include layout preservation, Arabic support and quality, translation accuracy and speed, pricing model, and Optical Character Recognition (OCR) support. To assess these tools, a series of test cases were conducted for each language, including: 1. Text -based documents: Evaluating basic translation accuracy, layout preservation, handling of number lists, bullet points, and right -to-left (RTL) and left -to-right (LTR) conversions. 2. Scanned documents: Testing OCR performance, particularly for Arabic, and pre

In [87]:
chunks_embed

{0: array([-0.07794044,  0.09288886, -0.05094891, -0.20239308, -0.21310513,
        -0.09023824, -0.23641534, -0.1556464 , -0.01257665, -0.04694957,
        -0.10418994, -0.18667635,  0.28209352, -0.08863356,  0.09518477,
        -0.21071735,  0.0704694 ,  0.26461634, -0.0455053 , -0.2260805 ,
         0.06006435,  0.18041575,  0.11362955, -0.15906906, -0.12787342,
        -0.19587693,  0.06354219, -0.09553141,  0.1106306 , -0.2014176 ,
         0.03607013,  0.57321584,  0.02429615,  0.11811383, -0.2011315 ,
         0.36605644, -0.1406599 ,  0.26839128,  0.0779558 ,  0.02715305,
        -0.1749437 ,  0.03605352,  0.0735817 ,  0.10512677, -0.06908429,
         0.08066356, -0.12097342,  0.21880853, -0.29462343,  0.15026909,
        -0.30775774, -0.08158635, -0.25272468,  0.1702018 ,  0.06833216,
        -0.35003015,  0.18344738,  0.23088385,  0.09044713, -0.08036122,
         0.07537191,  0.18127593, -0.27453184,  0.08772247, -0.12296198,
        -0.14061493,  0.0412959 ,  0.08955552, -

In [88]:
len(chunks_embed.keys())

16

## FAISS Implementation

In [89]:
def create_faiss_index(chunks_embed):
    """
    Create FAISS index from embeddings dictionary
    """
    # Convert dict to arrays
    indices = list(chunks_embed.keys())
    embeddings = np.array([chunks_embed[idx] for idx in indices]).astype('float32')
    
    # Create FAISS index
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    
    # Add embeddings
    index.add(embeddings)
    
    print(f"Created FAISS index with {index.ntotal} vectors of dimension {dimension}")
    return index, indices

index, indices = create_faiss_index(chunks_embed)   

Created FAISS index with 16 vectors of dimension 384


In [90]:
print(index)
print(indices)
print(f"Index type: {type(index)}")
print(f"Number of vectors: {index.ntotal}")
print(f"Vector dimension: {index.d}")


<faiss.swigfaiss.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x37bdfff60> >
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index type: <class 'faiss.swigfaiss.IndexFlatIP'>
Number of vectors: 16
Vector dimension: 384


In [91]:
def save_faiss_index(faiss_index, filepath,file_index_name):
    """Save FAISS index to disk"""
    faiss.write_index(faiss_index,f"{file_index_name}.faiss")
    print(f"Index saved to {filepath}")

def load_faiss_index(filepath, file_index_name):
    """Load FAISS index from disk"""
    index = faiss.read_index(f"{file_index_name}.faiss")
    print(f"Index loaded from {filepath}")
    return index

In [92]:
def search_faiss(faiss_index, index_mapping, query_embedding, top_k=5):
    """
    Search FAISS index
    """
    query_embedding = np.array([query_embedding]).astype('float32')
    distances, indices = faiss_index.search(query_embedding, top_k)
    
    results = []
    for i in range(top_k):
        faiss_idx = indices[0][i]
        if faiss_idx != -1:
            your_idx = index_mapping[faiss_idx]
            distance = distances[0][i]
            results.append((your_idx, distance))
    
    return results

In [93]:
def setup_faiss_search(chunks, embeddings_dict):
    """
    Set up FAISS search system

    """
    # Create chunks dictionary
    chunks_dict = {i: chunk for i, chunk in enumerate(chunks)}
    
    # Create FAISS index
    faiss_index, index_mapping = create_faiss_index(embeddings_dict)
    
    return faiss_index, index_mapping, chunks_dict

def search_chunks(faiss_index, index_mapping, chunks_dict, query_embedding, top_k=5):
    """
    Search for similar chunks
    """
    # Search FAISS
    results = search_faiss(faiss_index, index_mapping, query_embedding, top_k)
    
    # Format results
    formatted_results = []
    for chunk_idx, distance in results:
        formatted_results.append({
            'chunk_id': chunk_idx,
            'text': chunks_dict[chunk_idx],
            'distance': distance,
            'similarity': 1 / (1 + distance) 
        })
    
    return formatted_results

def search_with_text_query(faiss_index, index_mapping, chunks_dict, model, query_text, top_k=5):
    """
    Search using text query (encode query first)
    """
    query_embedding = model.encode(query_text)
    return search_chunks(faiss_index, index_mapping, chunks_dict, query_embedding, top_k)

In [94]:
faiss_index, index_mapping, chunks_dict = setup_faiss_search(chunks, chunks_embed)

Created FAISS index with 16 vectors of dimension 384


In [95]:
def quick_search_test(faiss_index, index_mapping, chunks_dict, model, prompt):
    """Quick test to verify search works"""
    query = prompt
    query_embedding = model.encode(query)
    results = search_faiss(faiss_index, index_mapping, query_embedding, top_k=3)
    
    print(f"Search results for '{query}':")
    for chunk_id, distance in results:
        chunk_text = chunks_dict[chunk_id]
        print(f"Chunk {chunk_id} (distance: {distance:.4f}): {chunk_text[:800]}...")
    
    return results

# Run test - you need to provide the prompt argument
quick_search_test(faiss_index, index_mapping, chunks_dict, model, "What are the recommended features to be added to the project?")

Search results for 'What are the recommended features to be added to the project?':
Chunk 8 (distance: 8.3162): These recommended features address critical gaps in current tools, such as limited editing capabilit ies, lack of flexible OCR options, insufficient support for domain -specific or tone - adjusted translations, absence of advanced translation modes, and the need for intuitive review interfaces. Implementing these enhancements could significantly improve the functionality and market competitiveness of document translation tools....
Chunk 4 (distance: 6.8217): Based on the evaluation, several opportunities exist to enhance document translation tools to better serve B2B and B2C markets. The following recommended features aim to address ga ps identified in the tested tools and improve functionality, user experience, and translation quality: • Editing Capabilities 1. PDF Editing: Enable users to add text, shapes, images, and freehand annotations to PDFs, facilitating document cust

[(8, 8.316237), (4, 6.8217278), (10, 5.7394834)]